In [1]:
%load_ext autoreload
import numpy as np
import pandas as pd

%autoreload 2
import zcu_tools.notebook.analysis.plot as zp
from zcu_tools.notebook.analysis.fluxdep import VisualizeSpet
from zcu_tools.notebook.persistance import load_result, load_spects
from zcu_tools.simulate import flx2mA, mA2flx
from zcu_tools.simulate.fluxonium import calculate_energy_vs_flx

In [2]:
qub_name = "Test049"

server_ip = "005-writeboard"
port = 4999

# Load data

## Parameters

In [ ]:
loadpath = f"../../result/{qub_name}/params.json"
_, params, mA_c, period, allows, _ = load_result(loadpath)
EJ, EC, EL = params

# mA_c = 4.46

print(allows)

if "r_f" in allows:
    r_f = allows["r_f"]

if "sample_f" in allows:
    sample_f = allows["sample_f"]


flxs = np.linspace(0.0, 1.0, 1000)
mAs = flx2mA(flxs, mA_c, period)

## Load Sample Points

In [ ]:
# loading points
loadpath = f"../../result/{qub_name}/freqs.csv"

freqs_df = pd.read_csv(loadpath)
s_mAs = freqs_df["Current (mA)"].values  # mA
s_fpts = freqs_df["Freq (MHz)"].values * 1e-3  # GHz
s_T1s = freqs_df["T1 (μs)"].values
s_T1errs = freqs_df["T1err (μs)"].values

# sort by flux
s_mAs, s_fpts, s_T1s, s_T1errs = tuple(
    np.array(a) for a in zip(*sorted(zip(s_mAs, s_fpts, s_T1s, s_T1errs)))
)
s_flxs = mA2flx(s_mAs, mA_c, period)

freqs_df.head(10)

# Matrix elements

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(5) if j > i]

fig = zp.plot_matrix_elements(params, flxs, show_idxs)
fig.show()

In [6]:
fig.write_html(f"../../result/{qub_name}/web/matrixelem.html", include_plotlyjs="cdn")
fig.write_image(f"../../result/{qub_name}/image/matrixelem.png", format="png")

# Flux dependence

In [ ]:
processed_spect_path = f"../../result/{qub_name}/spect.hdf5"
s_spects = load_spects(processed_spect_path)
s_spects.keys()

In [5]:
mA_c = list(s_spects.values())[-1]["mA_c"]
period = list(s_spects.values())[-1]["period"]
for spect in s_spects.values():
    shift = mA_c - spect["mA_c"]
    spect["mA_c"] += shift
    spect["spectrum"]["mAs"] += shift
    spect["points"]["mAs"] += shift

In [6]:
mA_bound = (
    np.nanmin([np.nanmin(s["spectrum"]["mAs"]) for s in s_spects.values()]),
    np.nanmax([np.nanmax(s["spectrum"]["mAs"]) for s in s_spects.values()]),
)
fpt_bound = (
    np.nanmin([np.nanmin(s["points"]["fpts"]) for s in s_spects.values()]),
    np.nanmax([np.nanmax(s["points"]["fpts"]) for s in s_spects.values()]),
)
s_selected = None
t_mAs = np.linspace(mA_bound[0], mA_bound[1], 1000)
t_fpts = np.linspace(fpt_bound[0], fpt_bound[1], 1000)
t_flxs = mA2flx(t_mAs, mA_c, period)

In [ ]:
_, energies = calculate_energy_vs_flx(params, t_flxs, cutoff=40, evals_count=15)

In [ ]:
v_allows = {
    **allows,
    # "transitions": [(i, j) for i in (0, 1, 2) for j in range(i + 1, 15)],
    # "red side": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "mirror": [(i, j) for i in (0, 1) for j in range(i + 1, 15)],
    # "mirror red": [(i, j) for i in (0, 1) for j in range(i + 1, 15)],
    "red side": [(0, 2)],
}

vs = VisualizeSpet(
    s_spects, np.array([]), np.array([]), t_mAs, energies, v_allows, auto_hide=False
)
fig = vs.create_figure()
_ = fig.update_layout(
    title=f"EJ/EC/EL = ({params[0]:.2f}, {params[1]:.2f}, {params[2]:.2f})",
    title_x=0.501,
)
# fig.update_yaxes(range=[allows["r_f"] - 0.01, allows["r_f"] + 0.01])
# fig.update_layout(height=1000)
fig.show()